# 1. Initializations

## 1.1 General imports

In [ ]:
### data management
import pandas as pd
import numpy as np

### régression
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict
from sklearn.feature_selection import f_regression

### graphical plotly basics
import matplotlib.pyplot as plt
%matplotlib inline

## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

## 1.3 General classification functions

In [ ]:
# None

# 2. Loading and Data Quality

## 2.1 Loading of data sets and general exploration

In [ ]:
df_auto_raw = dfc.load_dataset_from_config('auto_data', sep=',')

if df_auto_raw is not None and isinstance(df_auto_raw, pd.DataFrame):
    # display(df_auto_raw.head())
    dfc.log_general_info(df_auto_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_auto_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_auto_raw))
    df_auto = dfc.normalize_column_names(df_auto_raw)
    display(df_auto.head())

In [ ]:
df_auto_desc = df_auto.select_dtypes(include=np.number).describe()
display(df_auto_desc)
df_auto_cr = df_auto.select_dtypes(include=np.number).corr()
display(df_auto_cr)

## 2.2 Data quality refinement

In [ ]:
# Original backup and duplicates management
df_auto_orig = df_auto.copy()
df_auto = df_auto.drop_duplicates()
df_auto = df_auto[(df_auto.normalized_losses != '?') &
                  (df_auto.bore != '?') &
                  (df_auto.stroke != '?')]
df_auto.normalized_losses = df_auto.normalized_losses.astype(int)
df_auto.horsepower = df_auto.horsepower.astype(int)
df_auto.bore = df_auto.bore.astype(float)
df_auto.stroke = df_auto.stroke.astype(float)
df_auto.peak_rpm = df_auto.peak_rpm.astype(int)
df_auto.price = df_auto.price.astype(int)
df_auto = df_auto.select_dtypes(include=np.number)

In [ ]:
df_auto.info()
df_auto_desc = df_auto.select_dtypes(include=np.number).describe()
display(df_auto_desc)
df_auto_cr = df_auto.select_dtypes(include=np.number).corr()
display(df_auto_cr)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(df_auto['curb_weight'], df_auto['price'], color='darkblue');

# 2. Data Classification

## 2.1 General Analysis variable/target Separation

In [ ]:
# Separation des variables explicatives (features) et de la variable à prédire (target)
data = df_auto[['curb_weight']]
target = df_auto['price']

In [ ]:
# Séparation de données d'entrainement et données de test
X_train, X_test, y_train, y_test = train_test_split(data, target, train_size=0.8, random_state=126)

## 2.2 Linear Regression

In [ ]:
# Definition et Entrainement du modèle
regLR = LinearRegression()
regLR.fit(X_train, y_train)

In [ ]:
# Prédiction du modèle sur les données de test
y_pred = regLR.predict(X_test)

# Récupération des données d'ajustement pour une régression simple
print("l'ordonnée à l'origine calculée par le modèle:", regLR.intercept_)
print("la pente de la droite:",regLR.coef_)

In [ ]:
# Validation croisée entre 4 sous échantillons d'entrainement pour la régression (MSE / RMSE / MAE), test_score = R² par défaut
scores = cross_validate( # entraine et évalue le modèle sur chaque groupe (cv)
    regLR, X_train, y_train, 
    cv=4, return_train_score=True,
    scoring=['r2', 'neg_mean_squared_error','neg_root_mean_squared_error', 'neg_mean_absolute_error'])
print(
    f"Model: {regLR}\n"
    f"test Score (R²): {scores['test_r2'].mean().round(4)} "
    f"(+/- {scores['test_r2'].std().round(4)})\n"
    f"train Score (R²): {scores['train_r2'].mean().round(4)} "
    f"(+/- {scores['train_r2'].std().round(4)})\n"
    f"MSE Score: {scores['test_neg_mean_squared_error'].mean().round(0)} "
    f"(+/- {scores['test_neg_mean_squared_error'].std().round(0)})\n"
    f"RMSE Score: {scores['test_neg_root_mean_squared_error'].mean().round(0)} "
    f"(+/- {scores['test_neg_root_mean_squared_error'].std().round(0)})\n"
    f"MAE Score: {scores['test_neg_mean_absolute_error'].mean().round(0)} "
    f"(+/- {scores['test_neg_mean_absolute_error'].std().round(0)})"
)

In [ ]:
# Validation croisée avec prédiction via 4 sous échantillons sur les données d'entrainement
y_train_preds = cross_val_predict( # entraine et renvoie les prédictions sur chaque groupe (cv) considéré comme données de test
    regLR, X_train, y_train,
    cv=4)
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets)**2).mean())
print(f"RMSE Score: {rmse(y_train_preds, y_train).round(0)}")

In [ ]:
# Prédiction du modèle sur les données de test
y_pred = regLR.predict(X_test)

plt.figure(figsize=(8, 6))
plt.suptitle("Régression prédite appliquée sur l'échantillon de test")
plt.scatter(X_test, y_test, color='darkblue')
plt.plot(X_test, y_pred, color='black');

In [ ]:
# Analyse et affichage des résidus
y_residus = y_pred - y_test
y_residus_norm = (y_residus-y_residus.mean())/y_residus.std()

plt.figure(figsize=(8, 2))
plt.suptitle("Valeur des résidus en fonction de X_test")
plt.scatter(X_test, y_residus, color='red')
plt.plot([X_test.curb_weight.min(),X_test.curb_weight.max()], [0,0], color='black');

plt.figure(figsize=(8, 4))
plt.suptitle("Centrage et réduction des résidus et comparaison avec la bissectrice normale")
stats.probplot(y_residus_norm, plot=plt);

In [ ]:
# Evaluation du modèle sur les données de test
print("Score R² calculé par le modèle:", regLR.score(X_test, y_test))

In [ ]:
# Test statistique univarié sur chaque variable explicative de la cible (et sur les données totales)
# NB : cela ne prouve pas la causalité ni l'importance, juste la corrélation
X = df_auto.drop('price', axis=1)
y = df_auto['price']
f_statistics, p_values = f_regression(X, y)
for column, f, p in zip(X.columns, f_statistics, p_values):
    print (f"[{column}]\n [F-Stat : {f.round(2)}] [P-Value : {p.round(6)}]")